# Differential expression analysis
Data are GNPs from control, +Glu, and +Glu/+BAPTA samples.  
[DESeq2 docs](https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html)

## Some notes on results
**Adjusted p-value = NA.** By default DESEq2 does not perform multiple hypothesis correction for genes with low average expression in the data, under the assumption that genes with low expression are unlikely to be biologically relevant. See [DESeq2 note](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#indfilt) and [paper](https://www.pnas.org/doi/full/10.1073/pnas.0914005107). However, this results in different expression cutoffs for our comparisons of interest: 4 for ctrl vs. glu and 40 for glu vs bapta. Since we would prefer to test the same set of genes for both comparisons, we disable independent filtering here.  
**TODO**  
- make some plots

In [ ]:
Sys.setenv(LANGUAGE = "en") # set language to "ja" if you prefer
library(DESeq2)
library(dplyr)
library(tibble)
library(readr)

# Get and process your data

In [ ]:
data_location = 'data/counts_exon_uns.txt'
annotation_location = 'data/sample_metadata.txt'

as_matrix <- function(tibble){
    # Extract the first column and store it as row names
    row_names <- tibble[[1]]  # The 0th column (first column)
    # Remove the first column from the tibble
    my_tibble_no_first <- tibble %>% select(-1)
    # Convert the remaining tibble to a matrix
    my_matrix <- as.matrix(my_tibble_no_first)
    # Assign the first column as row names of the matrix
    rownames(my_matrix) <- row_names
    return(my_matrix)
}

# Read and format our annotation table
annot = read_table(annotation_location)

# Read and format our gene expression data
data = read_table(data_location,skip=1) %>% suppressWarnings %>%
    select(-Chr, -Start, -End, -Strand, -Length) %>%
    rename(!!!setNames(annot$file, annot$sample_name)) %>%
    as_matrix

# finish formatting our annotations...
annot <- annot %>% 
    select(-file) %>%
    as.data.frame
rownames(annot) <- annot[[1]]  # Set the first column as rownames
annot <- annot[, -1]           # Remove the first column from the data frame
annot[] <- lapply(annot, factor)
#annot$glu <- factor(annot$glu)
#coldata$type <- factor(coldata$type)

In [ ]:
data %>% head
annot %>% head

In [ ]:
get_hits <- function(deseq_result,decreasing=FALSE,outfile=NULL){
    res_df <- as.data.frame(deseq_result)  # Convert DESeqResults object to a data frame
    res_df <- res_df[order(res_df$log2FoldChange, decreasing=decreasing),]  # Sort by foldChange
    # Filter by padj < 0.05
    filtered_sorted_res <- res_df[(!is.na(res_df$padj)) & (res_df$padj < 0.05), ]  # Filter by padj
    
    if (!is.null(outfile)) {
        # Write the data frame to a tab-separated file (TSV)
        write.table(res_df, file = outfile, sep = "\t", quote = FALSE)
    }
    return(filtered_sorted_res)
}

# Full linear model
2 gnp, 2 glu, 2 bapta

In [ ]:
dds <- DESeq2::DESeqDataSetFromMatrix(countData = data,
                              colData = annot,
                              design = ~ glu + bapta)
# Pre-filter
smallestGroupSize <- 2
keep <- rowSums(counts(dds) >= 10) >= smallestGroupSize
dds <- dds[keep,]
dds$glu <- relevel(dds$glu, ref = TRUE)
# Run regression
dds <- DESeq(dds)
dds

In [ ]:
# A bit of QC - looks like replicates cluster together on PC1, which is good
vsd <- vst(dds, blind=TRUE)
plotPCA(vsd, intgroup=c("glu", "bapta"))

In [ ]:
## Generate DE comparisons. Wald test with BH correction. Post-hoc LFC shrinkage with apeglm.

# reference is glu+/bapta-, comparison is glu-/bapta-
res_wrt_glu_noshrink <- results(dds, contrast=c("glu",TRUE,FALSE),independentFiltering=FALSE)
res_wrt_glu <- lfcShrink(dds, coef="glu_TRUE_vs_FALSE", res=res_wrt_glu_noshrink, type="apeglm")

# reference is glu+/bapta-, comparison is glu+/bapta+
res_wrt_bapta_noshrink <- results(dds, contrast=c("bapta",TRUE,FALSE),independentFiltering=FALSE)
res_wrt_bapta <- lfcShrink(dds, coef="bapta_TRUE_vs_FALSE", res=res_wrt_bapta_noshrink, type="apeglm")

In [ ]:
# genes down in bapta+  
get_hits(res_wrt_bapta,FALSE,outfile='out/deg_wrt_bapta.tsv') %>% head(n=10)

In [ ]:
# genes up in glu+ relative to glu-
get_hits(res_wrt_glu,TRUE,outfile='out/deg_wrt_glu.tsv') %>% head(n=10)

In [ ]:
# Look at the result for a specific gene using this syntax
res_wrt_glu_noshrink[rownames(res_wrt_glu) == 'Fosl2',]
res_wrt_glu[rownames(res_wrt_glu) == 'Fosl2',]

res_wrt_bapta_noshrink[rownames(res_wrt_glu) == 'Fosl2',]
res_wrt_bapta[rownames(res_wrt_glu) == 'Fosl2',]

In [ ]:
metadata(res_wrt_glu)$filterThreshold
metadata(res_wrt_bapta)$filterThreshold

# Pairwise models
I don't know how to specify the design formula to answer glu+bapta- vs glu+bapta+, so let's create pairwise models of the comparisons of interest (and revisit the full linear model later).

In [ ]:
get_hits <- function(deseq_result,decreasing=FALSE){
    res_df <- as.data.frame(deseq_result)  # Convert DESeqResults object to a data frame
    # Filter by p_adjusted < 0.05 and sort by log2FoldChange
    filtered_sorted_res <- res_df[res_df$padj < 0.05, ]  # Filter by padj
    filtered_sorted_res <- filtered_sorted_res[order(filtered_sorted_res$log2FoldChange, decreasing=decreasing),]  # Sort by foldChange
    return(filtered_sorted_res)
}

## glu-/bapta- vs glu+/bapta-

In [ ]:
dds_glu <- DESeq2::DESeqDataSetFromMatrix(countData = data[,1:4],
                              colData = annot[1:4,1,drop = FALSE],
                              design = ~ glu)
# Pre-filter
smallestGroupSize <- 2
keep <- rowSums(counts(dds_glu) >= 10) >= smallestGroupSize
dds_glu <- dds_glu[keep,]
# Run regression
dds_glu <- DESeq(dds_glu)
dds_glu

In [ ]:
res_glu <- results(dds_glu)
get_hits(res_glu,TRUE) %>% head(n=10)
res_glu[rownames(res_glu) == 'Fos',]

## glu+/bapta- vs glu+/bapta+

In [ ]:
dds_bapta <- DESeq2::DESeqDataSetFromMatrix(countData = data[,3:6],
                              colData = annot[3:6,2,drop = FALSE],
                              design = ~ bapta)
smallestGroupSize <- 2
keep <- rowSums(counts(dds_bapta) >= 10) >= smallestGroupSize
dds_bapta <- dds_bapta[keep,]
# Run regression
dds_bapta <- DESeq(dds_bapta)
#dds_bapta
res_bapta <- results(dds_bapta)

In [ ]:
get_hits(res_bapta) %>% head(n=10)